<a href="https://colab.research.google.com/github/Akitsuyoshi/lora_sandbox/blob/main/lora_trainer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install transformers accelerate datasets evaluate peft

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 280.0/280.0 kB 2.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 536.7/536.7 kB 11.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 12.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 183.4/183.4 kB 14.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 15.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 18.6 MB/s eta 0:00:00



## data loading mrpc
---



In [2]:
from datasets import load_dataset

ds = load_dataset("glue", "mrpc")
ds

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Generating train split:   0%|          | 0/3668 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/408 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/1725 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['sentence1', 'sentence2', 'label', 'idx'],
        num_rows: 3668
    })
    validation: Dataset({
        features: ['sentence1', 'sentence2', 'label', 'idx'],
        num_rows: 408
    })
    test: Dataset({
        features: ['sentence1', 'sentence2', 'label', 'idx'],
        num_rows: 1725
    })
})

In [3]:
ds["train"][0], ds["validation"][0], ds["test"][0]

({'sentence1': 'Amrozi accused his brother , whom he called " the witness " , of deliberately distorting his evidence .',
  'sentence2': 'Referring to him as only " the witness " , Amrozi accused his brother of deliberately distorting his evidence .',
  'label': 1,
  'idx': 0},
 {'sentence1': "He said the foodservice pie business doesn 't fit the company 's long-term growth strategy .",
  'sentence2': '" The foodservice pie business does not fit our long-term growth strategy .',
  'label': 1,
  'idx': 9},
 {'sentence1': "PCCW 's chief operating officer , Mike Butcher , and Alex Arena , the chief financial officer , will report directly to Mr So .",
  'sentence2': 'Current Chief Operating Officer Mike Butcher and Group Chief Financial Officer Alex Arena will report to So .',
  'label': 1,
  'idx': 0})

In [4]:
from transformers import AutoTokenizer, DataCollatorWithPadding

checkpoint = "distilbert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(checkpoint, use_fast=True)

def tokenize_function(example):
  return tokenizer(example["sentence1"], example["sentence2"], truncation=True)

ds = ds.map(tokenize_function, batched=True)
# Comment out since dynamic padding is a default behavior in Trainer
# data_collator = DataCollatorWithPadding(tokenizer=tokenizer)
ds

tokenizer_config.json:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

Map:   0%|          | 0/3668 [00:00<?, ? examples/s]

Map:   0%|          | 0/408 [00:00<?, ? examples/s]

Map:   0%|          | 0/1725 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['sentence1', 'sentence2', 'label', 'idx', 'input_ids', 'attention_mask'],
        num_rows: 3668
    })
    validation: Dataset({
        features: ['sentence1', 'sentence2', 'label', 'idx', 'input_ids', 'attention_mask'],
        num_rows: 408
    })
    test: Dataset({
        features: ['sentence1', 'sentence2', 'label', 'idx', 'input_ids', 'attention_mask'],
        num_rows: 1725
    })
})

In [5]:
tokenizer.decode(ds["train"][0]["input_ids"]), tokenizer.decode(ds["validation"][0]["input_ids"])

('[CLS] amrozi accused his brother, whom he called " the witness ", of deliberately distorting his evidence. [SEP] referring to him as only " the witness ", amrozi accused his brother of deliberately distorting his evidence. [SEP]',
 '[CLS] he said the foodservice pie business doesn\'t fit the company\'s long - term growth strategy. [SEP] " the foodservice pie business does not fit our long - term growth strategy. [SEP]')

In [6]:
import numpy as np
import evaluate

metrics = evaluate.load("glue", "mrpc")
metrics.compute(predictions=[0, 1, 0, 1, 0], references=[0, 1, 1, 1, 1])

{'accuracy': 0.6, 'f1': 0.6666666666666666}

In [7]:
def compute_metrics(eval_preds):
  logits, labels = eval_preds
  predictions = np.argmax(logits, axis=-1)
  # print(predictions, labels)
  merics = evaluate.load("glue", "mrpc")
  return metrics.compute(predictions=predictions, references=labels)

compute_metrics(([[1, 0], [0, 1], [4, 1], [3, 4], [2, 1]], [0, 1, 1, 1, 1]))

{'accuracy': 0.6, 'f1': 0.6666666666666666}

In [8]:
from transformers import AutoModelForSequenceClassification

model = AutoModelForSequenceClassification.from_pretrained(checkpoint, num_labels=2)
model.classifier

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Linear(in_features=768, out_features=2, bias=True)

In [9]:
from transformers import Trainer

trainer = Trainer(
    model,
    eval_dataset=ds["test"],
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
)
init_eval = trainer.evaluate()
init_eval

{'eval_loss': 0.7055996656417847,
 'eval_accuracy': 0.3356521739130435,
 'eval_f1': 0.07431340872374798,
 'eval_runtime': 6.6932,
 'eval_samples_per_second': 257.724,
 'eval_steps_per_second': 32.272}

In [10]:
from peft import LoraConfig, get_peft_model

peft_config = LoraConfig(task_type="SEQ_CLS",
                         lora_dropout=0.1,
                         target_modules="all-linear") # all linear layers is trained
lora_model = get_peft_model(model, peft_config)
lora_model.print_trainable_parameters()

trainable params: 1,274,130 || all params: 68,247,588 || trainable%: 1.8669231211511828


In [11]:
from transformers import TrainingArguments

training_args = TrainingArguments("lora_trainer",
                                  evaluation_strategy="epoch",
                                  save_strategy="epoch",
                                  per_device_eval_batch_size=32,
                                  per_device_train_batch_size=32,
                                  num_train_epochs=6,
                                  weight_decay=0.01,
                                  label_smoothing_factor=0.01,
                                  load_best_model_at_end=True,
                                  fp16=True,
                                  )
lora_trainer = Trainer(
    lora_model,
    training_args,
    train_dataset=ds["train"],
    eval_dataset=ds["validation"],
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
)
lora_trainer.train()

Epoch,Training Loss,Validation Loss,Accuracy,F1
1,No log,0.596728,0.683824,0.812227
2,No log,0.549381,0.710784,0.816199
3,No log,0.510124,0.732843,0.822764
4,No log,0.497879,0.747549,0.833063
5,0.558600,0.478002,0.781863,0.849916
6,0.558600,0.475334,0.776961,0.847059


TrainOutput(global_step=690, training_loss=0.5391158836475317, metrics={'train_runtime': 95.1164, 'train_samples_per_second': 231.38, 'train_steps_per_second': 7.254, 'total_flos': 468933823767168.0, 'train_loss': 0.5391158836475317, 'epoch': 6.0})

In [12]:
lora_model_path = "distilbert_lora"
lora_model.save_pretrained(lora_model_path)
!ls {lora_model_path}

adapter_config.json  adapter_model.safetensors	README.md


In [13]:
from peft import AutoPeftModelForSequenceClassification

lora_model = AutoPeftModelForSequenceClassification.from_pretrained(lora_model_path)
lora_trainer = Trainer(
    lora_model,
    eval_dataset=ds["test"],
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
)
lora_eval = lora_trainer.evaluate()
lora_eval

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


{'eval_loss': 0.500485360622406,
 'eval_accuracy': 0.7576811594202899,
 'eval_f1': 0.8289689034369885,
 'eval_runtime': 4.2453,
 'eval_samples_per_second': 406.334,
 'eval_steps_per_second': 50.88}

In [14]:
import pandas as pd

pd.DataFrame([init_eval, lora_eval], index=["Untrained Model", "Trained Model"])[["eval_accuracy", "eval_f1"]]

,eval_accuracy,eval_f1
Untrained Model,0.335652,0.074313
Trained Model,0.757681,0.828969


In [17]:
# show full cell output
pd.set_option("display.max_colwidth", None)

sampled_testsets = ds["test"].shuffle().select(range(500))
df = pd.DataFrame(sampled_testsets)
df = df[["sentence1", "sentence2", "label"]]

predicitons = lora_trainer.predict(sampled_testsets)
df["predicted_label"] = np.argmax(predicitons[0], axis=-1)
df[df["label"]!=df["predicted_label"]].head()

,sentence1,sentence2,label,predicted_label
3,"Intel Corp. narrowed its second-quarter revenue forecast Thursday , saying sales of microprocessors were at the high end of normal while demand for communications chips remained soft .","Intel Corp. , the world 's biggest semiconductor maker , narrowed its second-quarter sales forecast as demand for microprocessors is reaching the high end of the company 's expectations .",0,1
6,A tropical storm rapidly developed in the Gulf of Mexico Sunday and was expected to hit somewhere along the Texas or Louisiana coasts by Monday night .,A tropical storm rapidly developed in the Gulf of Mexico on Sunday and could have hurricane-force winds when it hits land somewhere along the Louisiana coast Monday night .,0,1
14,"Omar 's brother , Zahid , and wife , Tahari , were arrested on May 2 in Derbyshire .","Zahid and Paveen Sharif were arrested in Derbyshire and Tabassum , Mr Sharif 's wife , in Nottinghamshire , last Friday .",0,1
18,Caldera acquired the Unix server software of the original SCO and changed its name to the SCO Group .,"SCO changed its name to Tarantella , and Caldera later changed its name to the SCO Group .",0,1
21,"The report forecasts there will be 71,079 hot spots worldwide this year , up from just 14,752 in 2002 and 1,214 in 2001 .","The report also claims that there will be up to 9.3 million visitors to hot spots this year , up again from the meagre 2.5 million in 2002 .",0,1
